In [19]:
from collections import defaultdict
from Openseas_Methods import pull_sales_data,make_graph,make_nft_graph,pull_nft_stats,edge_counts_to_percent
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import pygraphviz
import random
from pymongo import MongoClient
from tqdm import tqdm
import pandas as pd
from itertools import combinations
from collections import defaultdict
from datetime import datetime

In [ ]:
with open('combo_per.pkl','rb') as f:
    combos_per = pickle.load(f)

In [21]:

slug_to_counts = defaultdict(lambda: 0)
for key,val in combos_per.items():
    if key[0]=='apecoin' or key[1]=='apecoin':
        continue
    else:
        slug_to_counts[key[0]]+= val
        slug_to_counts[key[1]]+=val
top_1000 = dict(sorted(slug_to_counts.items(), key=lambda x: x[1], reverse=True)[:3000])
print(top_1000)

{'base-introduced': 537.5955632394648, 'gemesis': 344.6626467040258, 'usd-c': 202.24013262055317, 'rarible': 175.04584114332752, 'rainbow-zorb': 164.81023275169045, 'fundrop-1-mint-machine-by-numo': 154.40413894083648, 'eggi-nswap': 140.33938491256097, 'junglevibes-neymarjr': 124.76190821175199, 'unidentified-contract-kg9mf80eue': 124.34945502337344, 'known-origin': 119.07378746507906, 'undefined1231231': 114.19953382477007, 'uniswap': 113.98014563659106, '1inch-token': 110.73529411764709, 'fourimmortals': 106.78047865972503, 'the-escher-edition-1': 106.21570630331425, 'gms-reply-guys': 103.59630739908133, 'as-society-decays': 100.13088293778105, 'zerion-dna-onepointo': 99.86332265574178, 'shanghai-capella': 98.62270253764297, 'parallelalpha': 98.59393619291332, 'robocosmic': 97.47484839407863, 'merge-regenesis': 94.15382123939737, 'worst-city': 93.94733065734401, 'peacefulllove-io': 91.23324888098294, 'argus-genesis': 91.1470445324532, 'twilifeclub-image-collection-campaign': 88.78681

In [22]:
client = MongoClient()
db = client.NFTDB
stat_collection = db.NFTStats
valid_slugs = []
for key,val in top_1000.items():
    data = stat_collection.find_one({'slug':key})
    total_supply = int(data['total_supply'])
    total_volume = int(data['total_volume'])
    if total_supply<5 or total_supply>20000:
        continue
    if total_volume<30:
        continue
    else:
        valid_slugs.append(key)

In [23]:
print(len(valid_slugs))
# print(valid_slugs)

767


In [ ]:
date = "2023-01-01T11:59:59"
after = datetime.strptime(date,'%Y-%m-%dT%H:%M:%S')
for slug in valid_slugs:
    print(slug)
    pull_sales_data(collection_slug = slug,event_type='successful',after=after)

In [ ]:
from pymongo import MongoClient
client = MongoClient()
db =client.NFTDB
sales = db.salesCollection
bots = db.BotAddresses
bots = bots.find({},{'_id':1})
bot_list = list(bots)
bot_addresses = [list(x.values())[0] for x in bot_list]
addresses  = list(set(sales.distinct('buyer_address')+sales.distinct('seller_address')))
add_no_bot = [x for x in addresses if x not in bot_addresses]
add_no_bot.sort()
find_all_nfts_for_addresses(add_no_bot,dbname='NFTDB',API_KEY="fa400fba3fd44574b659bb0372b8b5d9")